In [ ]:
#@title Install and Auto restart
requirements_content = """accelerate==0.25.0
huggingface-hub==0.20.1
Pillow==10.1.0
transformers==4.36.2
einops==0.7.0
gradio==4.15.0
timm==0.9.12
pydub==0.25.1
edge-tts
#torch==2.1.2
#torchvision==0.16.2
"""
# Write the contents to the file
with open("/content/requirements.txt", "w") as file:
    file.write(requirements_content)

print(f"File '/content/requirements.txt' has been successfully created with the specified contents.")

from google.colab import output
output.eval_js('new Audio("https://github.com/neuralfalcon/Roop-Image-FaceSwap/raw/main/start.mp3").play()')
!pip install -r /content/requirements.txt
from IPython.display import clear_output
clear_output()
output.eval_js('new Audio("https://github.com/neuralfalcon/Roop-Image-FaceSwap/raw/main/install_voice.mp3").play()')
import time
time.sleep(6)
import os
os.kill(os.getpid(), 9)

In [1]:
#@title <-- Play the audio { display-mode: "form" }

# !git clone https://github.com/vikhyat/moondream.git
# %cd /content/moondream
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from PIL import Image

# model_id = "vikhyatk/moondream2"
# revision = "2024-03-13"
# model = AutoModelForCausalLM.from_pretrained(
#     model_id, trust_remote_code=True, revision=revision
# )
# tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)
# from PIL import Image
# image = Image.open('/content/monalisa.jpg')
# image = Image.open('/content/monalisa.jpg')
# enc_image = model.encode_image(image)
# print(model.answer_question(enc_image, "Describe this image.", tokenizer))
%%html
<b>Press play on the music player to keep the tab alive, then run the cell below</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [8]:
#@title <-- Just run the cell (config edge TTS)
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list




import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path


text = 'This is Moondream Demo'  # @param {type: "string"}
Language = "English" # @param ['English']

Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global long_sentence,translate_text_flag,Language,speed,voice_name
  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path

yes_path=talk("yes")
clear_output()
edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

['This is Moondream Demo']
edge-tts  --rate=+0% --voice en-US-AriaNeural --text "This is Moondream Demo" --write-media /content/audio/e27cee98.mp3


In [6]:
#@title Downlod and Config MoonDream [If you are from the future change latest date in the revision variable to get the latest model]
import torch
import re
import time
# from moondream.moondream import detect_device, LATEST_REVISION
from threading import Thread
from transformers import TextIteratorStreamer, AutoTokenizer, AutoModelForCausalLM
from PIL import Image


import torch
if torch.cuda.is_available():
    use_cpu=False
else:
    use_cpu=True
# Specify whether to use CPU or GPU
# use_cpu = True  # Change to True if you want to use CPU

if use_cpu:
  device = torch.device("cpu")
  dtype = torch.float32
else:
  device=torch.device("cuda")
  dtype=torch.float16
print("Using device:", device)

revision = "2024-03-13"
model_id = "vikhyatk/moondream2"
# tokenizer = AutoTokenizer.from_pretrained(model_id, revision=LATEST_REVISION)
# moondream = AutoModelForCausalLM.from_pretrained(
#     model_id, trust_remote_code=True, revision=LATEST_REVISION
# ).to(device=device, dtype=dtype)

tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)

moondream = AutoModelForCausalLM.from_pretrained(
    model_id, trust_remote_code=True, revision=revision
).to(device=device, dtype=dtype)
moondream.eval()
from IPython.display import clear_output
clear_output()

In [ ]:
#@title Gradio webcam
import gradio as gr
from IPython.display import Audio, display
def answer_question(img, prompt):
    image_embeds = moondream.encode_image(img)
    streamer = TextIteratorStreamer(tokenizer, skip_special_tokens=True)
    thread = Thread(
        target=moondream.answer_question,
        kwargs={
            "image_embeds": image_embeds,
            "question": prompt,
            "tokenizer": tokenizer,
            "streamer": streamer,
        },
    )
    thread.start()

    buffer = ""
    for new_text in streamer:
        clean_text = re.sub("<$|END$", "", new_text)
        buffer += clean_text
        yield buffer.strip("<END")
default_prompt = "Is the person is standing? Please response with 'yes' or 'no'"  # @param {type: "string"}
audio_alert = True #@param {type:"boolean"}
with gr.Blocks() as demo:
    gr.Markdown("# 🌔 moondream")

    gr.HTML(
        """
        <style type="text/css">
            .md_output p {
                padding-top: 1rem;
                font-size: 1.2rem !important;
            }
        </style>
        """
    )

    with gr.Row():
        prompt = gr.Textbox(
            label="Prompt",
            value=default_prompt,
            interactive=True,
        )
    with gr.Row():
        img = gr.Image(type="pil", label="Upload an Image", streaming=True)
        output = gr.Markdown(elem_classes=["md_output"])

    latest_img = None
    latest_prompt = prompt.value

    @img.change(inputs=[img])
    def img_change(img):
        global latest_img
        latest_img = img

    @prompt.change(inputs=[prompt])
    def prompt_change(prompt):
        global latest_prompt
        latest_prompt = prompt

    @demo.load(outputs=[output])
    def live_video():
        global yes_path
        while True:
            if latest_img is None:
                time.sleep(0.1)
            else:
                predicted_result = []
                for text in answer_question(latest_img, latest_prompt):
                    if len(text) > 0:
                        predicted_result.append(text)

                predicted_result = '\n'.join(predicted_result)
                if predicted_result.lower().strip()=="yes":
                  if audio_alert:
                    display(Audio(yes_path, autoplay=True))
                yield predicted_result
                # for text in answer_question(latest_img, latest_prompt):
                #     if len(text) > 0:
                #         yield text


demo.launch(debug=True)
# demo.queue().launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://dbdd96efe491fbf6cf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
